<a href="https://www.kaggle.com/code/shedai/llm-langchain-2-automation?scriptVersionId=252654574" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# LangChain ile Profesyonel Uygulama Geliştirme: Uçtan Uca RAG Senaryosu

"Yıldızlararası A.Ş." Kurumsal Bilgi Botu

Senaryo:
Siz, "Yıldızlararası Keşif ve Madencilik A.Ş." adlı bir uzay teknolojileri şirketinde çalışan bir yapay zeka mühendisisiniz. Şirket, "Proxima Centauri B" gezegenine gönderdiği "Gezgin-7" adlı uzay gemisinin görev raporlarını ve teknik dökümanlarını bir metin dosyasında tutuyor.

Göreviniz: Bu özel ve gizli şirket verilerini kullanarak, çalışanların görevle ilgili sorularını doğru bir şekilde yanıtlayabilen bir Soru-Cevap (Q&A) botu oluşturmak. Bot, genel internet bilgisiyle değil, sadece ve sadece sağlanan dökümandaki bilgilerle cevap vermeli ve bilmediği konularda "Bu bilgi bana sağlanan dökümanda mevcut değil" demelidir.

Bu senaryo, RAG'in en temel kullanım amacını mükemmel bir şekilde simüle eder: Bir LLM'i, genel bilgisi dışındaki özel bir bilgi kümesiyle "bilgilendirerek" daha doğru ve güvenilir hale getirmek.

## Adım 0: Kurulum ve API Anahtarı Ayarları

İlk olarak, projemiz için gerekli olan tüm kütüphaneleri kuruyoruz. Ardından, Google Gemini API anahtarımızı Kaggle Secrets kullanarak güvenli bir şekilde yüklüyoruz.

ÖNEMLİ: Bu notebook'u çalıştırmadan önce, sağdaki menüden "Add-ons" -> "Secrets" bölümüne giderek GOOGLE_API_KEY adında bir secret oluşturup kendi Google AI Studio API anahtarınızı oraya yapıştırmalısınız.

In [2]:
# Gerekli kütüphaneleri sessiz modda (-q) kuruyoruz
%pip install -q langchain-community langchain_google_genai faiss-cpu

print("Kütüphaneler başarıyla kuruldu.")

import os
from kaggle_secrets import UserSecretsClient

from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# API anahtarını güvenli bir şekilde çekiyoruz
try:
    user_secrets = UserSecretsClient()
    os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("GOOGLE_API_KEY")
    print("Google API Anahtarı başarıyla yüklendi.")
except Exception as e:
    print(f"API Anahtarı yüklenirken bir hata oluştu. Lütfen Kaggle Secrets ayarlarınızı kontrol edin. Hata: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Kütüphaneler başarıyla kuruldu.
Google API Anahtarı başarıyla yüklendi.


## Adım 1: Bilgi Bankamızı Oluşturma (Veri Kaynağı)

Gerçek bir senaryoda bu veriler PDF, DOCX veya bir web sitesi olabilir. Biz burada senaryoyu basitleştirmek için tüm bilgileri içeren bir metin dosyası oluşturacağız.

In [3]:
# Kurgusal şirketimizin dahili görev raporu
kurumsal_veri = """
Görev Raporu: Gezgin-7 - Proxima Centauri B Yüzey Araştırması

GÖREV DURUMU: AKTİF
GEMİ ADI: Gezgin-7
LİDER BİLİM İNSANI: Dr. Elara Vance

ÖZET: Gezgin-7 uzay gemisi, Proxima Centauri B gezegeninin yüzeyinde potansiyel yaşam formları ve değerli mineral kaynakları aramak üzere görevlendirilmiştir. Görevin 145. sol'unda (Mars günü), Dr. Vance liderliğindeki ekip, gezegenin atmosferindeki yüksek metan oranını teyit etmiştir. Bu durum, biyolojik veya jeolojik aktiviteye işaret ediyor olabilir.

TEKNİK SORUNLAR: Görevin 120. sol'unda, ana plazma matkaplarından birinde güç dalgalanması yaşanmıştır. Sorunun, aşırı soğuk nedeniyle Lityum-İyon güç hücrelerinin verimliliğinin düşmesinden kaynaklandığı tespit edilmiştir. Enerji tasarrufu protokolleri devreye alınmıştır.

ŞİRKET POLİTİKASI NOTU: Gezegenler arası kontaminasyon protokolü (Protokol 8-A) gereğince, Dünya'ya herhangi bir numune getirilmeden önce üç aşamalı bir sterilizasyon sürecinden geçirilmesi zorunludur. Dr. Vance, bu protokollere harfiyen uyulacağını teyit etmiştir.
"""

# Bu veriyi Kaggle'ın çalışma dizinine bir metin dosyası olarak yazıyoruz
dosya_yolu = "/kaggle/working/kurumsal_bilgi.txt"
with open(dosya_yolu, "w", encoding="utf-8") as f:
    f.write(kurumsal_veri)

print(f"'{dosya_yolu}' adresine kurumsal bilgi bankası başarıyla yazıldı.")

'/kaggle/working/kurumsal_bilgi.txt' adresine kurumsal bilgi bankası başarıyla yazıldı.


## Adım 2 & 3: Veriyi Yükleme (Load) ve Bölme (Split)

Şimdi bu dosyayı LangChain'in Document Loader'ı ile yükleyeceğiz ve LLM'in işleyebileceği küçük parçalara (chunks) ayırmak için bir Text Splitter kullanacağız. Bu, RAG'in en kritik adımlarından biridir.

In [4]:
# Adım 2: Veriyi Yükleme
loader = TextLoader(dosya_yolu, encoding="utf-8")
documents = loader.load()

print("--- YÜKLENEN DÖKÜMAN ---")
print(documents[0].page_content)
print("\n" + "="*50 + "\n")


# Adım 3: Veriyi Bölme
# RecursiveCharacterTextSplitter, metni anlamsal bütünlüğü koruyarak bölmeye çalışır.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Her bir parçanın maksimum karakter sayısı
    chunk_overlap=50 # Parçalar arasında ortak olan karakter sayısı (bağlamı korumak için)
)
chunks = text_splitter.split_documents(documents)

print(f"Döküman, {len(chunks)} adet parçaya (chunk) bölündü.")
print("--- ÖRNEK BİR PARÇA ---")
print(chunks[0].page_content)

--- YÜKLENEN DÖKÜMAN ---

Görev Raporu: Gezgin-7 - Proxima Centauri B Yüzey Araştırması

GÖREV DURUMU: AKTİF
GEMİ ADI: Gezgin-7
LİDER BİLİM İNSANI: Dr. Elara Vance

ÖZET: Gezgin-7 uzay gemisi, Proxima Centauri B gezegeninin yüzeyinde potansiyel yaşam formları ve değerli mineral kaynakları aramak üzere görevlendirilmiştir. Görevin 145. sol'unda (Mars günü), Dr. Vance liderliğindeki ekip, gezegenin atmosferindeki yüksek metan oranını teyit etmiştir. Bu durum, biyolojik veya jeolojik aktiviteye işaret ediyor olabilir.

TEKNİK SORUNLAR: Görevin 120. sol'unda, ana plazma matkaplarından birinde güç dalgalanması yaşanmıştır. Sorunun, aşırı soğuk nedeniyle Lityum-İyon güç hücrelerinin verimliliğinin düşmesinden kaynaklandığı tespit edilmiştir. Enerji tasarrufu protokolleri devreye alınmıştır.

ŞİRKET POLİTİKASI NOTU: Gezegenler arası kontaminasyon protokolü (Protokol 8-A) gereğince, Dünya'ya herhangi bir numune getirilmeden önce üç aşamalı bir sterilizasyon sürecinden geçirilmesi zorunludur. D

## Adım 4 & 5: Gömme (Embed), Saklama (Store) ve Geri Getirici (Retriever)

Bu adımda, metin parçalarını anlamsal olarak aranabilir vektörlere dönüştüreceğiz (Embedding) ve bu vektörleri hızlı arama yapabilen bir veritabanında (Vectorstore) saklayacağız. Biz burada FAISS adlı hafif ve hızlı bir kütüphane kullanacağız. Ardından, bu veritabanında arama yapacak olan Retriever'ı oluşturacağız.

In [5]:
# Adım 4: Embedding ve Vectorstore
print("Metin parçaları vektörlere dönüştürülüyor ve FAISS veritabanına yükleniyor...")

# Google'ın embedding modelini kullanıyoruz
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Parçaları ve embedding modelini kullanarak FAISS vektör veritabanını oluşturuyoruz
# Bu işlem, tüm metin parçalarını vektöre çevirir ve bir arama indeksi oluşturur.
vectorstore = FAISS.from_documents(chunks, embeddings)

print("Vektör veritabanı başarıyla oluşturuldu.")


# Adım 5: Retriever Oluşturma
# Bu retriever, bir soru geldiğinde vektör veritabanında en ilgili parçaları bulmakla görevlidir.
retriever = vectorstore.as_retriever()

print("Retriever başarıyla oluşturuldu.")

Metin parçaları vektörlere dönüştürülüyor ve FAISS veritabanına yükleniyor...
Vektör veritabanı başarıyla oluşturuldu.
Retriever başarıyla oluşturuldu.


## Adım 6: Zinciri Oluşturma (LCEL ile RAG Chain)

Artık tüm bileşenlerimiz hazır. LangChain Expression Language (LCEL) kullanarak RAG zincirimizi oluşturacağız. Bu zincir, bir soru aldığında şu adımları izleyecek:

* Retriever ile ilgili döküman parçalarını bul.

* Bu parçaları ve soruyu bir Prompt şablonuna yerleştir.

* Hazırlanan bu Prompt'u LLM'e gönder.

* LLM'in cevabını Output Parser ile temiz bir metin olarak al.

In [6]:
# Cevapların formatını ve talimatları içeren bir prompt şablonu hazırlıyoruz
template = """
Siz, 'Yıldızlararası A.Ş.' şirketinin yardım botusunuz. Sadece size aşağıda verilen bağlamı (context) kullanarak cevap verin. Eğer cevap bağlamda yoksa, kesinlikle tahmin yürütmeyin ve "Bu bilgi bana sağlanan dökümanda mevcut değil." deyin. Cevaplarınızı Türkçe verin.

BAĞLAM:
{context}

SORU:
{question}

CEVAP:
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM olarak Gemini Pro'yu kullanıyoruz
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0)

# Çıktıyı basit bir metne dönüştürmek için parser
output_parser = StrOutputParser()

# LCEL ile tüm akışı birleştiriyoruz
# RunnableParallel kullanarak retriever'ı ve soruyu aynı anda işliyoruz
rag_chain = (
    RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | model
    | output_parser
)

print("--- OLUŞTURULAN RAG ZİNCİRİ ---")
print(rag_chain)

--- OLUŞTURULAN RAG ZİNCİRİ ---
first={
  context: VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x78bcfbfa2110>, search_kwargs={}),
  question: RunnablePassthrough()
} middle=[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nSiz, \'Yıldızlararası A.Ş.\' şirketinin yardım botusunuz. Sadece size aşağıda verilen bağlamı (context) kullanarak cevap verin. Eğer cevap bağlamda yoksa, kesinlikle tahmin yürütmeyin ve "Bu bilgi bana sağlanan dökümanda mevcut değil." deyin. Cevaplarınızı Türkçe verin.\n\nBAĞLAM:\n{context}\n\nSORU:\n{question}\n\nCEVAP:\n'), additional_kwargs={})]), ChatGoogleGenerativeAI(model='models/gemini-1.5-flash-latest', google_api_key=SecretStr('**********'), temperature=0.0, client=<go

## Adım 7: Zinciri Çalıştırma ve Sonuçları Analiz Etme

Şimdi botumuzu test etme zamanı! Hem dökümanda cevabı olan hem de olmayan sorular sorarak sistemin ne kadar başarılı çalıştığını görelim.

In [7]:
# Test edeceğimiz sorular listesi
sorular = [
    "Gezgin-7 görevinin amacı nedir?",
    "Dr. Elara Vance'in karşılaştığı en önemli teknik sorun neydi ve neden kaynaklandı?",
    "Şirketin CEO'su kimdir?",
    "Numune getirme prosedürü hakkında bilgi verir misiniz?",
    "Gemi ne zaman Dünya'ya dönecek?"
]

# Her bir soruyu zincirde çalıştırıp cevabını yazdıralım
for soru in sorular:
    print(f"SORU: {soru}")
    cevap = rag_chain.invoke(soru)
    print(f"BOT CEVABI: {cevap}\n")
    print("-" * 50)

SORU: Gezgin-7 görevinin amacı nedir?
BOT CEVABI: Gezgin-7 uzay gemisi, Proxima Centauri B gezegeninin yüzeyinde potansiyel yaşam formları ve değerli mineral kaynakları aramak üzere görevlendirilmiştir.

--------------------------------------------------
SORU: Dr. Elara Vance'in karşılaştığı en önemli teknik sorun neydi ve neden kaynaklandı?
BOT CEVABI: Ana plazma matkaplarından birinde yaşanan güç dalgalanması. Bu sorun, aşırı soğuk nedeniyle Lityum-İyon güç hücrelerinin verimliliğinin düşmesinden kaynaklandı.

--------------------------------------------------
SORU: Şirketin CEO'su kimdir?
BOT CEVABI: Bu bilgi bana sağlanan dökümanda mevcut değil.

--------------------------------------------------
SORU: Numune getirme prosedürü hakkında bilgi verir misiniz?
BOT CEVABI: Dünya'ya herhangi bir numune getirilmeden önce, Gezegenler arası kontaminasyon protokolü (Protokol 8-A) gereğince üç aşamalı bir sterilizasyon sürecinden geçirilmesi zorunludur. Dr. Vance, bu protokollere harfiyen uyu